# Step indexed logical relations

https://www.ccs.neu.edu/home/amal/papers/lr-recquant-techrpt.pdf amal ahmed  Step-Indexed Syntactic Logical Relations
for Recursive and Quantified Types 2006

Mcallester and appell https://dl.acm.org/doi/10.1145/504709.504712 An indexed model of recursive types for foundational proof-carrying code 2001

Type rules with |= rather than |-. Yes.

https://www.ccs.neu.edu/home/amal/papers/impred.pdf An Indexed Model of Impredicative Polymorphism and Mutable References

Semantic Typing - associate type to set of denotational semantic things
Miler 78 - denotational semantics.

https://www.cs.uoregon.edu/research/summerschool/summer24/lectures/Ahmed.pdf Semantic Type Soundness and Language Interoperability. OPLSS 24

https://www.andrewwagner.io/assets/slides/sem-tech-tut-prl.pdf  Semantic Typing and Separation Logic a tutorial Wagner



Step indexing is kind of defining the semantics notion of equality/equivalence by type.

in what sense _is_   {True : True, False : False} === {0 : 0, 1 : 1}. You have to define it. not obvious.

E and V
E is the set of expressions, V is set of values.

https://cstheory.stackexchange.com/questions/5427/what-are-the-differences-between-logical-relations-and-simulations Interesting connection. 


There's a difference between `(a -> Bool)` and `[a]`  reps of sets. Nat as (a -> Bool) is fine, but non terminating as a generator.

We can't have _just_ the checker version though because we need to supply functions with examples.
Two step indices?


idea: memoize results internally so if we ask for n again, we don't recompute? Seems fine.
idea2: take has_type examples and insert them into the generator. Known examples are probably useful. There's something cute there.




## Gluing
https://proofassistants.stackexchange.com/questions/32/what-is-artin-gluing-and-how-is-it-useful-in-proving-meta-theoretic-properties
https://arxiv.org/pdf/2010.08599  Logical Relations as Types
https://golem.ph.utexas.edu/category/2013/04/scones_logical_relations_and_p.html 




## Is type soundness interesting?
https://blog.sigplan.org/2019/10/17/what-type-soundness-theorem-do-you-really-want-to-prove/

https://cs.stackexchange.com/questions/138835/whats-the-relationship-between-semantic-type-soundness-and-functional-correc
`type Type = MemState, int -> frozenset` python models of ref cell ML?

https://www.seas.upenn.edu/~sweirich/papers/liu-mltt-consistency.pdf Functional Pearl: Short and Mechanized Logical Relation for
Dependent Type Theories




https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/10351.BentonNick.Paper_.2808.pdf Step-Indexing: The Good, the Bad and the Ugly

https://arxiv.org/abs/1304.7857  A Step-Indexing Approach to Partial Functions. Slind. ACL2 Interesting. Is this the same step indexing? Not really types?

https://link.springer.com/chapter/10.1007/978-3-540-32033-3_29 A Tutorial Example of the Semantic Approach to Foundational Proof-Carrying Code https://www.site.uottawa.ca/~afelty/dist/rta05.pdf  A Tutorial Example of the Semantic Approach to
Foundational Proof-Carrying Code

# Modal
https://xavierleroy.org/CdF/2018-2019/8.pdf leroy notes. Later modality

https://www.cs.princeton.edu/~appel/papers/modalmodel.pdf A Very Modal Model of a Modern, Major, General Type System. moadlity. https://dl.acm.org/doi/10.1145/1190215.1190235  

https://people.mpi-sws.org/~dreyer/papers/lslr/old-lmcs.pdf LOGICAL STEP-INDEXED LOGICAL RELATIONS DEREK DREYER, AMAL AHMED, AND LARS BIRKEDAL

Lob induction






# Python

In [ ]:
# Growing set model. step indexing?
# [Inj,Set] has decidable equality.
# aka a generator or stream.
from typing import Callable, Union, List, Tuple, Dict, Any
type Type = Callable[[int], frozenset]
from frozendict import frozendict
import itertools

Bool = lambda n: frozenset([False, True])
Void = lambda n: frozenset()
Nat = lambda n: frozenset(range(n))


# Indices vs parameters show up in a difference of where I need to decrease n?

type Family = Callable[[object], Type] # or is there more data? Guarantees about int? Do int come in?
def Sigma(A : Type, B : Family):
    return lambda n: frozenset((a,b) for a in A(n-1) for b in B(a)(n-1)) # Do I need n-1 here or would n be fine? n//2 some other function? (n - m, m) pairs?
def Pi(A : Type, B : Family) -> Type:
    def res(n):
        Alist = list(A(n-1))
        return frozenset(frozendict({k:v for k,v in zip(Alist, bvs)}) for bvs in itertools.product(*[B(a)(n-1) for a in Alist]))
    return res
# Non dependent versions may not need to consume gas?

def Vec(A : Type, m : int) -> Type:
    return lambda n: frozenset(itertools.product(A(n), repeat=m))

def List(A : Type) -> Type:
    return lambda n: frozenset([()]) if n == 0 else frozenset(itertools.product(A(n), repeat=n)) | List(A)(n-1)


List(Bool)(6)

# inductive datatypes
def List1(A : Type) -> Type:
    return lambda n: frozenset() if n == 0 else frozenset([("nil",)]) | frozenset([ ("cons", hd, tl) for tl in List1(A)(n-1) for hd in A(n)])
List1(Bool)(3)

def Fix(f : Callable[[Type], Type]) -> Type:
    def res(n):
        return frozenset() if n == 0 else f(Fix(f)(n-1))
    return res

List2 = lambda A: Fix(lambda L: lambda n: frozenset([("nil",)]) | frozenset(("cons", A(n), tl) for tl in L(n-1)))

List2(Bool)(3)


def Id(A, x, y):
    return lambda n: frozenset([x]) if x == y else frozenset()

def is_type(A):
    acc = A(0)
    for n in range(4):
        assert isinstance(acc, frozenset), f"Expected a frozenset at n={n}, got {type(acc)}"
        s = A(n)
        assert is_subset(s, acc)
        acc = s

def has_type(t, A):


def Univ(l : int) -> Type:
    def res(n):
        if l == 0:
            return frozenset([Bool, Void, Nat]) | Univ(Pi(A, B) for ...) # But they are unhashable...
        if n == 0:
            return frozenset()
        return Univ(l-1)(n) | 




https://docs.python.org/3/library/sys.monitoring.html  For tracking number of steps
Hmm. maybe I can `set_local_events` . The number of instructions doesn't seem that reliable. Ok. maybe it's alright.



In [14]:
import sys
TOOLID = 3
sys.monitoring.use_tool_id(TOOLID, "steplimiter")
steps = 10000
def cb(code, offset):
    global steps
    steps -= 1
    if steps <= 0:
        steps = 10000000 # Other stuff is happening?
        raise RuntimeError("Step limit exceeded")
sys.monitoring.register_callback(TOOLID, sys.monitoring.events.INSTRUCTION, cb)
sys.monitoring.set_events(TOOLID, sys.monitoring.events.INSTRUCTION)
try:
    while True:
        print(steps)
finally:
    sys.monitoring.register_callback(TOOLID, sys.monitoring.events.INSTRUCTION, None)
    print("Step limit reached, monitoring stopped.")
    sys.monitoring.register_callback(
            TOOLID, sys.monitoring.events.INSTRUCTION, None
        )
    sys.monitoring.free_tool_id(TOOLID)


9280
8613
8181
7749
7317
6885
6453
6021
5589
5157
4725
4293
3861
3429
2997
2565
2133
1701
1269
837
405
Step limit reached, monitoring stopped.


RuntimeError: Step limit exceeded

In [14]:
import types
isinstance(lambda x: x, types.CodeType)
f = lambda x: x
isinstance(f.__code__, types.CodeType)
exec(f.__code__, locals={"x" : 42})

TypeError: 'locals' is an invalid keyword argument for exec()

Reals. n can give a 1/n ball size or 2^-n + [-N, N] bounding box.
 

In [ ]:
import fractions
import flint


#flint.arb(n) for range(n)
def I(n):
    return frozenset([fractions.Fraction(i, n) for i in range(n+1)])
I(10)


frozenset({Fraction(0, 1),
           Fraction(1, 10),
           Fraction(1, 5),
           Fraction(3, 10),
           Fraction(2, 5),
           Fraction(1, 2),
           Fraction(3, 5),
           Fraction(7, 10),
           Fraction(4, 5),
           Fraction(9, 10),
           Fraction(1, 1)})

In [5]:
import mpmath
def I(n):
    with mpmath.workdps(n):
        return frozenset([mpmath.mpf(i)/n for i in range(n+1)])
I(3)


frozenset({mpf('0.0'),
           mpf('0.33331298828125'),
           mpf('0.6666259765625'),
           mpf('1.0')})

In [ ]:
set([flint.arb(1)]) # can't hash....

NameError: name 'flint' is not defined